# 1  读取数据，随机打乱

In [1]:
import os
import random

def get_dataset():
    data = []
    for root, dirs, files in os.walk(r'E:\mix20_rand700_tokens_cleaned\tokens\neg'):
        for file in files:
            realpath = os.path.join(root, file)
            with open(realpath, errors='ignore') as f:
                data.append((f.read(), 'bad'))
    for root, dirs, files in os.walk(r'E:\mix20_rand700_tokens_cleaned\tokens\pos'):
        for file in files:
            realpath = os.path.join(root, file)
            with open(realpath, errors='ignore') as f:
                data.append((f.read(), 'good'))
    random.shuffle(data)

    return data

In [2]:
data = get_dataset()

# 2  将数据集划分为10份

In [3]:
train_data = []
train_target = []
test_data = []
test_target = []

def train_and_test_data(data_):
    filesize = int(0.1 * len(data_))
    # 训练集和测试集的比例为9:1
    for i in range(10):
        train_data_ = [each[0] for each in data_[filesize*i : filesize*(i+1)]]
        train_target_ = [each[1] for each in data_[filesize*i : filesize*(i+1)]]

        test_data_ = [each[0] for each in data_[0:filesize*i] + data_[filesize*(i+1) :]]
        test_target_ = [each[1] for each in data_[0:filesize*i] + data_[filesize*(i+1) :]]

        train_data.append(train_data_)
        train_target.append(train_target_)
        test_data.append(test_data_)
        test_target.append(test_target_)

# 3  定义预测函数

In [4]:
# 多项式朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB

def nbc_classfier1(train_data,train_target,test_data,test_target):
    nbc = Pipeline([('vect', TfidfVectorizer()),('clf', MultinomialNB(alpha=1.0)),])
    nbc.fit(train_data, train_target) # 训练多项式朴素贝叶斯分类器
    predict = nbc.predict(test_data)  # 在测试集上预测结果
    count = 0 # 统计预测正确的结果个数
    for left , right in zip(predict, test_target):
          if left == right:
                count += 1
    result = float(count/len(test_target))
    return result

In [5]:
# 伯努利朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB

def nbc_classfier2(train_data,train_target,test_data,test_target):
    nbc= Pipeline([('vect', TfidfVectorizer()),('clf', BernoulliNB(alpha=0.1)),])
    nbc.fit(train_data, train_target) # 训练伯努利朴素贝叶斯分类器
    predict = nbc.predict(test_data) # 在测试集上预测结果
    count = 0 # 统计预测正确的结果个数
    for left , right in zip(predict, test_target):
          if left == right:
                count += 1
    result = float(count/len(test_target))
    return result

# 4 进行预测

In [ ]:
train_and_test_data(data)

result_multi = 0.0
for i in range(10):
    result = 0.0
    result = nbc_classfier1(train_data[i],train_target[i],test_data[i],test_target[i])
    result_multi = result_multi + result

result_multi = result_multi/10.0

result_berno = 0.0
for j in range(10):
    result = 0.0
    result = nbc_classfier2(train_data[j],train_target[j],test_data[j],test_target[j])
    result_berno = result_berno + result

result_berno = result_berno/10.0

print(result_multi)
print(result_berno)